In [56]:
#Imports e configuração de caminho
import sys, os
import pandas as pd
import numpy as np

# Adiciona src/ ao path
src_path = os.path.join(os.getcwd(), "src")
sys.path.append(src_path)

from fetchers import fetch_inmet_climate, fetch_ibge_population_density
from src.utils import normalize_column_names


# Cria pastas de saída
os.makedirs("data/raw", exist_ok=True)


In [57]:
#Carregamento da base de dados
dengue_url = ("https://raw.githubusercontent.com/valdineyatilio/ProjetoAplicado-IV/main/A1/DadosDengueCSV.csv")
df_dengue = pd.read_csv(dengue_url)
df_dengue = normalize_column_names(df_dengue)
df_dengue.to_csv("data/raw/dengue.csv", index=False)

print("Base de dengue salva:", df_dengue.shape)


Base de dengue salva: (812, 30)


In [58]:
#Coleta dados climáticos do INMET ou gera simulação
stations = ["A507", "A518", "A541"]
try:
    df_clima = fetch_inmet_climate(stations, "2015-01-01", "2025-12-31")
    if df_clima.empty:
        raise RuntimeError("Dados vazios")
    print("Clima INMET coletado:", df_clima.shape)
except:
    print("Dados INMET indisponíveis. Gerando simulação.")
    weeks = pd.date_range("2018-01-01", "2023-12-31", freq="W-MON")
    df_clima = pd.DataFrame({
        "datahora": weeks,
        "station": "SIMULADA",
        "year": weeks.isocalendar().year,
        "week": weeks.isocalendar().week,
        "t_mean": np.random.normal(22, 3, len(weeks)),
        "precip_sum": np.random.gamma(2, 10, len(weeks))
    })

df_clima = normalize_column_names(df_clima)
df_clima.to_csv("data/raw/clima_inmet.csv", index=False)


[WARN] Estação A507 retornou status 404. Pulando.
[WARN] Estação A518 retornou status 404. Pulando.
[WARN] Estação A541 retornou status 404. Pulando.
[WARN] Nenhuma estação retornou dados válidos.
Dados INMET indisponíveis. Gerando simulação.


In [60]:
#Coleta de dados populacionais
try:
    df_pop = fetch_ibge_population_density()
    df_pop = normalize_column_names(df_pop)
    df_pop.to_csv("data/raw/pop_density.csv", index=False)
    print("População IBGE salva:", df_pop.shape)
except Exception as e:
    print(f" Erro ao coletar dados populacionais: {e}")

 Erro ao coletar dados populacionais: fetch_ibge_population_density() missing 2 required positional arguments: 'municipio_ids' and 'year'


In [61]:
#Conclusão
print("Coleta de dados concluída com sucesso.")
print("Arquivos salvos em data/raw/:")
print("- dengue.csv")
print("- clima_inmet.csv")
print("- pop_density.csv")

Coleta de dados concluída com sucesso.
Arquivos salvos em data/raw/:
- dengue.csv
- clima_inmet.csv
- pop_density.csv


In [62]:
#Arquivos Gerados
#- data/raw/dengue.csv → base original do InfoDengue
# data/raw/clima_inmet.csv → dados climáticos simulados
# data/raw/pop_density.csv → densidade populacional simulada
